# Imports and Data Loading

In [ ]:
import os, pandas as pd, numpy as np

In [ ]:
docking_results_path = "/Users/alexpayne/Scientific_Projects/local_test/mers_hallucination_hybrid/"

In [ ]:
csv_name = "all_results.csv"

In [ ]:
csv_path = os.path.join(docking_results_path, csv_name)

In [ ]:
os.path.exists(csv_path)

In [ ]:
df = pd.read_csv(csv_path)

In [ ]:
df.head()

In [ ]:
df

In [ ]:
df

# Basic Plotting

In [ ]:
import plotly.express as px

## RMSD vs Chemgauss4

In [ ]:
px.density_heatmap(df, 
           x="docked_RMSD", 
           y="chemgauss4_score", 
#            color="POSIT_prob", 
#            hover_data=["MERS_structure", 
#                        "SARS_ligand", 
#                        ], 
          )

# How many structures < 2 Å do we have per compound?

### don't understand how this lambda function works but it does seem to work

In [ ]:
RMSD_gt0 = df.groupby('SARS_ligand')[['docked_RMSD']].apply(lambda x: x[(x > 0)].count())
RMSD_gt0_lt2 = df.groupby('SARS_ligand')[['docked_RMSD']].apply(lambda x: x[(x > 0) & (x <= 2)].count())
RMSD_mean = df.groupby('SARS_ligand')[['docked_RMSD']].apply(lambda x: x[(x > 0)].mean())
RMSD_min = df.groupby('SARS_ligand')[['docked_RMSD']].apply(lambda x: x[(x > 0)].min())
compound_df = pd.concat([RMSD_gt0, RMSD_gt0_lt2, RMSD_mean, RMSD_min], axis=1)
compound_df.columns = ["Count_Calc_RMSD", "Count_Good_RMSD", "Mean", "Min"]
compound_df["Compound ID"] = compound_df.index

In [ ]:
compound_df

In [ ]:
RMSD_min = df.groupby('SARS_ligand')[['docked_RMSD']].apply(lambda x: x[(x > 0)].min())

In [ ]:
RMSD_min

In [ ]:
set(RMSD_min.MERS_structure)

In [ ]:
compound_df.head()

In [ ]:
compound_df[compound_df.Count_Calc_RMSD == 0]

In [ ]:
compound_df[compound_df.Count_Good_RMSD == 0]

In [ ]:
compound_df[compound_df.Count_Calc_RMSD == 10]

## For how many compounds are we just not able to calculate RMSD?

In [ ]:
fig = px.bar(compound_df.sort_values("Count_Calc_RMSD"), y='Count_Calc_RMSD')
fig.update_traces(width=1)
fig.update_layout(height=800, 
                  width=12600, 
                  font_size=12,
#                  xaxis={'categoryorder':'category ascending'}
                 )
fig.show()
fig.write_image("../figures/compounds_missing_RMSD_calcs.png")

In [ ]:
px.histogram(compound_df, x='Count_Calc_RMSD')

In [ ]:
px.histogram(compound_df, x='Count_Good_RMSD')

In [ ]:
px.histogram(compound_df, x='Mean')

### How many < 2Å structures per compound?

In [ ]:
fig = px.bar(compound_df.sort_values("Count_Good_RMSD"), y="Count_Good_RMSD")
fig.update_traces(width=1)
fig.update_layout(height=600, 
                  width=12600, 
                  font_size=12,
#                  xaxis={'categoryorder':'category ascending'}
                 )
fig.show()
fig.write_image("../figures/how_many_successful_complexes_per_compound_id.png")

## Average RMSD per compound?

In [ ]:
fig = px.bar(compound_df.sort_values("Mean"), y="Mean")
fig.update_traces(width=1)
fig.update_layout(height=600, 
                  width=12600, 
                  font_size=12,
#                  xaxis={'categoryorder':'category ascending'}
                 )
fig.show()
fig.write_image("../figures/avg_rmsd_per_compound_id.png")

## Scatter plot

In [ ]:
fig = px.scatter(compound_df,
          x="Mean",
          y="Count_Good_RMSD",
           color="Count_Calc_RMSD",
          hover_data=["Compound ID"],
          color_continuous_scale="Viridis_R")
fig.update_layout(height=800,
                 width=1200,
                 font_size=16)
fig.show()
fig.write_image("../figures/compound_results_scatterplot.png")

## analyzing specific examples

In [ ]:
RMSD_count[RMSD_count.index == 'Mpro-z7vh8_0A_bound']

# Try Per Compound Analysis with API

In [ ]:
sys.path.append("../")
from asapdiscovery.docking import analysis as a
from importlib import reload

In [ ]:
reload(a)

In [ ]:
dr = a.DockingResults("/Users/alexpayne/Scientific_Projects/local_test/mers_hallucination_hybrid/all_results.csv")
dr.df = dr.df.drop("Unnamed: 0", axis=1)
dr.df["MERS_structure"] = dr.df["MERS_structure"].apply(lambda x: x.split("_")[1])
dr.df

In [ ]:
dr.df.columns = ["Compound_ID", "Structure_Source", "Docked_File", "RMSD", "POSIT", "Chemgauss4"]

In [ ]:
dr.df

In [ ]:
dr.get_grouped_df(groupby_ID_column="Compound_ID")

In [ ]:
dr.get_compound_df()

In [ ]:
dr.get_structure_df()

In [ ]:
dr.structure_df

In [ ]:
px.scatter(dr.compound_df,
          x="Mean_RMSD",
          y="Mean_Chemgauss4")

In [ ]:
px.scatter(dr.compound_df,
          x="Min_RMSD",
          y="Min_Chemgauss4")

In [ ]:
px.scatter(dr.structure_df,
          x="Min_RMSD",
          y="Min_Chemgauss4",
          color="Resolution")

In [ ]:
px.scatter(dr.structure_df,
          x="Min_RMSD",
          y="Min_Chemgauss4",
          color="Structure_Source")

In [ ]:
px.scatter(dr.compound_df,
           x="Not_NA_RMSD",
           y="Not_NA_Chemgauss4",)

In [ ]:
px.histogram(dr.compound_df,
            x="Not_NA_RMSD")

## test lambda function

In [ ]:
compound_ID_column = "Compound_ID"


def get_n_good_score(feature):
    if feature == "RMSD":
        lambda_func = lambda x: x[(x <= 2)].count()
    elif feature == "POSIT":
        lambda_func = lambda x: x[(x > 0.7)].count()
    elif feature == "Chemgauss4":
        lambda_func = lambda x: x[(x < 0)].count()
    else:
        raise NotImplementedError(f"good score acquisition not implemented for {feature}")
    return lambda_func
# get_good_score(feature)
for feature in ["RMSD", "POSIT", "Chemgauss4"]:
    print(dr.df.groupby(compound_ID_column)[[feature]].apply(get_good_score(feature)).head())

# Were some MERS structures better than others?

In [ ]:
# PDB_structure_mean = df[['docked_RMSD', 'MERS_structure',]].groupby('MERS_structure').mean()
PDB_structure_mean = df.groupby('MERS_structure')[['docked_RMSD']].apply(lambda x: x[(x > 0)].mean())
# PDB_structure_total = df[['docked_RMSD', 'MERS_structure',]].groupby('MERS_structure').count()
PDB_structure_count = df.groupby('MERS_structure')[['docked_RMSD']].apply(lambda x: x[(x > 0) & (x <= 2)].count())

# RMSD_gt0_lt2 = df.groupby('SARS_ligand')[['docked_RMSD']].apply(lambda x: x[(x > 0) & (x <= 2)].count())
# RMSD_mean = df.groupby('SARS_ligand')[['docked_RMSD']].apply(lambda x: x[(x > 0)].mean())
structure_df = pd.concat([PDB_structure_mean, PDB_structure_count], axis=1)
structure_df.columns = ["Mean", "Count_Good_RMSD"]
structure_df["PDB_ID"] = [pdb_id.split("_")[1] for pdb_id in structure_df.index]
structure_df.index = structure_df.PDB_ID

In [ ]:
structure_df

In [ ]:
with open("../scripts/mers_structures.csv") as f:
    mers_structure_df = pd.read_csv(f)

In [ ]:
mers_structure_df

In [ ]:
structure_df["Resolution"] = list(mers_structure_df.Resolution)

In [ ]:
structure_df

In [ ]:
px.scatter(structure_df,
          x='Mean',
          y='Count_Good_RMSD',
           color="Resolution",
          hover_data=["Resolution"]
          )

In [ ]:
px.scatter(structure_df,
          x='Resolution',
          y='Count_Good_RMSD',
           color="Mean",
          hover_data=["Resolution"]
          )

In [ ]:
px.bar(structure_df.sort_values("Mean"),
       y='Mean',
      )

# test importing pickle

In [ ]:
import pickle as pkl

In [ ]:
with open(os.path.join(docking_results_path, "AAR-POS-0daf6b7e-10_0A/rcsb_4RSP/results.pkl"), "rb") as f:
    results = pkl.load(f)

In [ ]:
results

# Test if using NAN instead of -1 gives nice results

In [ ]:
nandf = df.fillna(float(-1.0))

In [ ]:
nandf

In [ ]:
type(list(set(df.POSIT_prob))[0])

## nvm can't get it to work